In [1]:
from __future__ import print_function
import paddle
import paddle.fluid as fluid
import numpy as np
import sys
import math

CLASS_DIM = 2 #Number of categories for sentiment analysis
EMB_DIM = 64 #Dimensions of the word vector
HID_DIM = 64 #Dimensions of hide layer
STACKED_NUM = 2 #LSTM Layers of the bidirectional stack
BATCH_SIZE = 64 #batch size
EPOCH = 20
LEARNING_RATE=0.0001

In [2]:
#Textconvolution neural network
def convolution_net(data, input_dim, class_dim, emb_dim, hid_dim):
    emb = fluid.layers.embedding(
        input=data, size=[input_dim, emb_dim], is_sparse=True)
    conv_3 = fluid.nets.sequence_conv_pool(
        input=emb,
        num_filters=hid_dim,
        filter_size=3,
        act="tanh",
        pool_type="sqrt")
    conv_4 = fluid.nets.sequence_conv_pool(
        input=emb,
        num_filters=hid_dim,
        filter_size=4,
        act="tanh",
        pool_type="sqrt")
    prediction = fluid.layers.fc(
        input=[conv_3, conv_4], size=class_dim, act="softmax")
    return prediction

In [3]:
#Stack Bidirectional LSTM
def stacked_lstm_net(data, input_dim, class_dim, emb_dim, hid_dim, stacked_num):

    # Calculate word vectorvector
    emb = fluid.layers.embedding(input=data, size=[input_dim, emb_dim], is_sparse=True)

    #First stack
    #Fully connected layer
    fc1 = fluid.layers.fc(input=emb, size=hid_dim)
    #lstm layer
    lstm1, cell1 = fluid.layers.dynamic_lstm(input=fc1, size=hid_dim)

    inputs = [fc1, lstm1]

    #All remaining stack structures
    for i in range(2, stacked_num + 1):
        fc = fluid.layers.fc(input=inputs, size=hid_dim)
        lstm, cell = fluid.layers.dynamic_lstm(input=fc, size=hid_dim, is_reverse=(i % 2) == 0)
        inputs = [fc, lstm]

    #pooling layer
    fc_last = fluid.layers.sequence_pool(input=inputs[0], pool_type='max')
    lstm_last = fluid.layers.sequence_pool(input=inputs[1], pool_type='max')

    #Fully connected layer, softmax prediction
    prediction = fluid.layers.fc(
        input=[fc_last, lstm_last], size=class_dim, act='softmax')
    return prediction

In [4]:
def inference_program(word_dict):
    data = fluid.layers.data(
        name="words", shape=[1], dtype="int64", lod_level=1)

    dict_dim = len(word_dict)
#     net = convolution_net(data, dict_dim, CLASS_DIM, EMB_DIM, HID_DIM)
    net = stacked_lstm_net(data, dict_dim, CLASS_DIM, EMB_DIM, HID_DIM, STACKED_NUM)
    return net

In [5]:
def train_program(prediction):
    label = fluid.layers.data(name="label", shape=[1], dtype="int64")
    cost = fluid.layers.cross_entropy(input=prediction, label=label)
    avg_cost = fluid.layers.mean(cost)
    accuracy = fluid.layers.accuracy(input=prediction, label=label)
    return [avg_cost, accuracy] #return average cost and accuracy acc

#Optimization function
def optimizer_func():
#     return fluid.optimizer.Adagrad(learning_rate=LEARNING_RATE)
    return fluid.optimizer.Adam(learning_rate=LEARNING_RATE)

In [6]:
use_cuda = True 
place = fluid.CUDAPlace(1) if use_cuda else fluid.CPUPlace()

In [7]:
print("Loading IMDB word dict....")
word_dict = paddle.dataset.imdb.word_dict()

print ("Reading training data....")
train_reader = paddle.batch(
    paddle.reader.shuffle(
        paddle.dataset.imdb.train(word_dict), buf_size=25000),
    batch_size=BATCH_SIZE)
print("Reading testing data....")
test_reader = paddle.batch(
    paddle.dataset.imdb.test(word_dict), batch_size=BATCH_SIZE)

Loading IMDB word dict....
Reading training data....
Reading testing data....


In [8]:
len(word_dict)

5147

In [9]:
exe = fluid.Executor(place)
prediction = inference_program(word_dict)
[avg_cost, accuracy] = train_program(prediction)#training program
sgd_optimizer = optimizer_func()# training optimization function
sgd_optimizer.minimize(avg_cost)

([inputs {
    parameter: "Beta1Pow"
    arguments: "embedding_0.w_0_beta1_pow_acc_0"
  }
  inputs {
    parameter: "Beta1Tensor"
  }
  inputs {
    parameter: "Beta2Pow"
    arguments: "embedding_0.w_0_beta2_pow_acc_0"
  }
  inputs {
    parameter: "Beta2Tensor"
  }
  inputs {
    parameter: "Grad"
    arguments: "embedding_0.w_0@GRAD"
  }
  inputs {
    parameter: "LearningRate"
    arguments: "learning_rate_0"
  }
  inputs {
    parameter: "Moment1"
    arguments: "embedding_0.w_0_moment1_0"
  }
  inputs {
    parameter: "Moment2"
    arguments: "embedding_0.w_0_moment2_0"
  }
  inputs {
    parameter: "Param"
    arguments: "embedding_0.w_0"
  }
  outputs {
    parameter: "Beta1PowOut"
    arguments: "embedding_0.w_0_beta1_pow_acc_0"
  }
  outputs {
    parameter: "Beta2PowOut"
    arguments: "embedding_0.w_0_beta2_pow_acc_0"
  }
  outputs {
    parameter: "Moment1Out"
    arguments: "embedding_0.w_0_moment1_0"
  }
  outputs {
    parameter: "Moment2Out"
    arguments: "embedding_0

In [10]:
def train_test(program, reader):
    count = 0
    feed_var_list = [
        program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder_test = fluid.DataFeeder(feed_list=feed_var_list, place=place)
    test_exe = fluid.Executor(place)
    accumulated = len([avg_cost, accuracy]) * [0]
    for test_data in reader():
        avg_cost_np = test_exe.run(
            program=program,
            feed=feeder_test.feed(test_data),
            fetch_list=[avg_cost, accuracy])
        accumulated = [
            x[0] + x[1][0] for x in zip(accumulated, avg_cost_np)
        ]
        count += 1
    return [x / count for x in accumulated]

In [11]:
import time

# Specify the directory path to save the parameters
params_dirname = "model/sentiment_lstm.inference.model"
RESULT_FILE = 'result/results_lstm.txt'

feed_order = ['words', 'label']

# Main loop part of the program
def train_loop(main_program):
    # Start the trainer built above
    exe.run(fluid.default_startup_program())

    feed_var_list_loop = [
        main_program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder = fluid.DataFeeder(
        feed_list=feed_var_list_loop, place=place)

    test_program = fluid.default_main_program().clone(for_test=True)

    # Training loop
    start_time = time.time()
    for epoch_id in range(EPOCH):
        for step_id, data in enumerate(train_reader()):
            # Running trainer
            metrics = exe.run(main_program,
                              feed=feeder.feed(data),
                              fetch_list=[avg_cost, accuracy])


            print("Step {0}, Epoch {1} Metrics {2}".format(
                step_id, epoch_id, list(map(np.array,
                                            metrics))))

#             if step_id == 30:
#                 if params_dirname is not None:
#                     fluid.io.save_inference_model(params_dirname, ["words"],
#                                                   prediction, exe)# Save model
#                 return
            
    total_time = time.time() - start_time
    with open(RESULT_FILE, 'a') as f:
        f.write('\n\n\ntraining results: \n')
        f.write('\n total training time: \t {}'.format(total_time))
#         f.write('\n final average training lostt: \t %.3f' % (average_loss))

    start_time = time.time()
    # Testing Results
    avg_cost_test, acc_test = train_test(test_program, test_reader)
    s = 'Step {0}, Test Loss {1:0.2}, Acc {2:0.2}'.format(step_id, avg_cost_test, acc_test)
    print(s)
    with open(RESULT_FILE, 'a') as f:
        f.write(s)

In [12]:
train_loop(fluid.default_main_program())

Step 0, Epoch 0 Metrics [array([0.6912526], dtype=float32), array([0.515625], dtype=float32)]
Step 1, Epoch 0 Metrics [array([0.6919825], dtype=float32), array([0.515625], dtype=float32)]
Step 2, Epoch 0 Metrics [array([0.6947759], dtype=float32), array([0.46875], dtype=float32)]
Step 3, Epoch 0 Metrics [array([0.6865495], dtype=float32), array([0.5625], dtype=float32)]
Step 4, Epoch 0 Metrics [array([0.6924192], dtype=float32), array([0.5], dtype=float32)]
Step 5, Epoch 0 Metrics [array([0.68527734], dtype=float32), array([0.59375], dtype=float32)]
Step 6, Epoch 0 Metrics [array([0.69498694], dtype=float32), array([0.46875], dtype=float32)]
Step 7, Epoch 0 Metrics [array([0.6928976], dtype=float32), array([0.5], dtype=float32)]
Step 8, Epoch 0 Metrics [array([0.6899418], dtype=float32), array([0.515625], dtype=float32)]
Step 9, Epoch 0 Metrics [array([0.686556], dtype=float32), array([0.59375], dtype=float32)]
Step 10, Epoch 0 Metrics [array([0.6938534], dtype=float32), array([0.48437

Step 89, Epoch 0 Metrics [array([0.681309], dtype=float32), array([0.75], dtype=float32)]
Step 90, Epoch 0 Metrics [array([0.6823417], dtype=float32), array([0.734375], dtype=float32)]
Step 91, Epoch 0 Metrics [array([0.6771939], dtype=float32), array([0.796875], dtype=float32)]
Step 92, Epoch 0 Metrics [array([0.68154275], dtype=float32), array([0.75], dtype=float32)]
Step 93, Epoch 0 Metrics [array([0.678138], dtype=float32), array([0.75], dtype=float32)]
Step 94, Epoch 0 Metrics [array([0.6790589], dtype=float32), array([0.78125], dtype=float32)]
Step 95, Epoch 0 Metrics [array([0.68121094], dtype=float32), array([0.78125], dtype=float32)]
Step 96, Epoch 0 Metrics [array([0.68239975], dtype=float32), array([0.71875], dtype=float32)]
Step 97, Epoch 0 Metrics [array([0.6809647], dtype=float32), array([0.734375], dtype=float32)]
Step 98, Epoch 0 Metrics [array([0.6800293], dtype=float32), array([0.703125], dtype=float32)]
Step 99, Epoch 0 Metrics [array([0.67888814], dtype=float32), ar

Step 178, Epoch 0 Metrics [array([0.6651054], dtype=float32), array([0.765625], dtype=float32)]
Step 179, Epoch 0 Metrics [array([0.6632261], dtype=float32), array([0.78125], dtype=float32)]
Step 180, Epoch 0 Metrics [array([0.6593093], dtype=float32), array([0.75], dtype=float32)]
Step 181, Epoch 0 Metrics [array([0.6532625], dtype=float32), array([0.84375], dtype=float32)]
Step 182, Epoch 0 Metrics [array([0.6647049], dtype=float32), array([0.78125], dtype=float32)]
Step 183, Epoch 0 Metrics [array([0.6532931], dtype=float32), array([0.8125], dtype=float32)]
Step 184, Epoch 0 Metrics [array([0.6641068], dtype=float32), array([0.765625], dtype=float32)]
Step 185, Epoch 0 Metrics [array([0.6637559], dtype=float32), array([0.75], dtype=float32)]
Step 186, Epoch 0 Metrics [array([0.6582102], dtype=float32), array([0.8125], dtype=float32)]
Step 187, Epoch 0 Metrics [array([0.65492797], dtype=float32), array([0.828125], dtype=float32)]
Step 188, Epoch 0 Metrics [array([0.6493198], dtype=fl

Step 265, Epoch 0 Metrics [array([0.62869495], dtype=float32), array([0.765625], dtype=float32)]
Step 266, Epoch 0 Metrics [array([0.6221794], dtype=float32), array([0.8125], dtype=float32)]
Step 267, Epoch 0 Metrics [array([0.6245876], dtype=float32), array([0.8125], dtype=float32)]
Step 268, Epoch 0 Metrics [array([0.6359043], dtype=float32), array([0.703125], dtype=float32)]
Step 269, Epoch 0 Metrics [array([0.6174139], dtype=float32), array([0.78125], dtype=float32)]
Step 270, Epoch 0 Metrics [array([0.6274402], dtype=float32), array([0.765625], dtype=float32)]
Step 271, Epoch 0 Metrics [array([0.62705135], dtype=float32), array([0.734375], dtype=float32)]
Step 272, Epoch 0 Metrics [array([0.634246], dtype=float32), array([0.6875], dtype=float32)]
Step 273, Epoch 0 Metrics [array([0.60292673], dtype=float32), array([0.8125], dtype=float32)]
Step 274, Epoch 0 Metrics [array([0.61786234], dtype=float32), array([0.796875], dtype=float32)]
Step 275, Epoch 0 Metrics [array([0.6303083], 

Step 351, Epoch 0 Metrics [array([0.5671066], dtype=float32), array([0.78125], dtype=float32)]
Step 352, Epoch 0 Metrics [array([0.5442845], dtype=float32), array([0.828125], dtype=float32)]
Step 353, Epoch 0 Metrics [array([0.5483892], dtype=float32), array([0.8125], dtype=float32)]
Step 354, Epoch 0 Metrics [array([0.55738145], dtype=float32), array([0.8125], dtype=float32)]
Step 355, Epoch 0 Metrics [array([0.5830728], dtype=float32), array([0.796875], dtype=float32)]
Step 356, Epoch 0 Metrics [array([0.5603322], dtype=float32), array([0.78125], dtype=float32)]
Step 357, Epoch 0 Metrics [array([0.61642563], dtype=float32), array([0.6875], dtype=float32)]
Step 358, Epoch 0 Metrics [array([0.54267585], dtype=float32), array([0.84375], dtype=float32)]
Step 359, Epoch 0 Metrics [array([0.5675988], dtype=float32), array([0.765625], dtype=float32)]
Step 360, Epoch 0 Metrics [array([0.5355227], dtype=float32), array([0.921875], dtype=float32)]
Step 361, Epoch 0 Metrics [array([0.5784428], 

...

Step 389, Epoch 18 Metrics [array([0.1794377], dtype=float32), array([0.953125], dtype=float32)]
Step 390, Epoch 18 Metrics [array([0.06358447], dtype=float32), array([0.975], dtype=float32)]
Step 0, Epoch 19 Metrics [array([0.07540184], dtype=float32), array([0.953125], dtype=float32)]
Step 1, Epoch 19 Metrics [array([0.10775843], dtype=float32), array([0.953125], dtype=float32)]
Step 2, Epoch 19 Metrics [array([0.13934663], dtype=float32), array([0.953125], dtype=float32)]
Step 3, Epoch 19 Metrics [array([0.12529595], dtype=float32), array([0.96875], dtype=float32)]
Step 4, Epoch 19 Metrics [array([0.0948758], dtype=float32), array([0.984375], dtype=float32)]
Step 5, Epoch 19 Metrics [array([0.09366256], dtype=float32), array([0.984375], dtype=float32)]
Step 6, Epoch 19 Metrics [array([0.18164936], dtype=float32), array([0.953125], dtype=float32)]
Step 7, Epoch 19 Metrics [array([0.06473657], dtype=float32), array([0.984375], dtype=float32)]
Step 8, Epoch 19 Metrics [array([0.1216494

Step 85, Epoch 19 Metrics [array([0.13462132], dtype=float32), array([0.96875], dtype=float32)]
Step 86, Epoch 19 Metrics [array([0.07259309], dtype=float32), array([0.984375], dtype=float32)]
Step 87, Epoch 19 Metrics [array([0.12325969], dtype=float32), array([0.96875], dtype=float32)]
Step 88, Epoch 19 Metrics [array([0.2314758], dtype=float32), array([0.921875], dtype=float32)]
Step 89, Epoch 19 Metrics [array([0.09808296], dtype=float32), array([0.96875], dtype=float32)]
Step 90, Epoch 19 Metrics [array([0.12903094], dtype=float32), array([0.96875], dtype=float32)]
Step 91, Epoch 19 Metrics [array([0.07591004], dtype=float32), array([1.], dtype=float32)]
Step 92, Epoch 19 Metrics [array([0.1147116], dtype=float32), array([0.953125], dtype=float32)]
Step 93, Epoch 19 Metrics [array([0.15909109], dtype=float32), array([0.90625], dtype=float32)]
Step 94, Epoch 19 Metrics [array([0.07345086], dtype=float32), array([1.], dtype=float32)]
Step 95, Epoch 19 Metrics [array([0.10850859], dt

Step 171, Epoch 19 Metrics [array([0.06894398], dtype=float32), array([0.96875], dtype=float32)]
Step 172, Epoch 19 Metrics [array([0.11511579], dtype=float32), array([0.953125], dtype=float32)]
Step 173, Epoch 19 Metrics [array([0.12671477], dtype=float32), array([0.953125], dtype=float32)]
Step 174, Epoch 19 Metrics [array([0.09308854], dtype=float32), array([0.96875], dtype=float32)]
Step 175, Epoch 19 Metrics [array([0.07445619], dtype=float32), array([0.984375], dtype=float32)]
Step 176, Epoch 19 Metrics [array([0.11212001], dtype=float32), array([0.953125], dtype=float32)]
Step 177, Epoch 19 Metrics [array([0.17915857], dtype=float32), array([0.9375], dtype=float32)]
Step 178, Epoch 19 Metrics [array([0.09452544], dtype=float32), array([0.953125], dtype=float32)]
Step 179, Epoch 19 Metrics [array([0.12665558], dtype=float32), array([0.953125], dtype=float32)]
Step 180, Epoch 19 Metrics [array([0.1586352], dtype=float32), array([0.9375], dtype=float32)]
Step 181, Epoch 19 Metrics 

Step 258, Epoch 19 Metrics [array([0.06908067], dtype=float32), array([1.], dtype=float32)]
Step 259, Epoch 19 Metrics [array([0.16136953], dtype=float32), array([0.953125], dtype=float32)]
Step 260, Epoch 19 Metrics [array([0.09664781], dtype=float32), array([0.953125], dtype=float32)]
Step 261, Epoch 19 Metrics [array([0.08132839], dtype=float32), array([0.96875], dtype=float32)]
Step 262, Epoch 19 Metrics [array([0.21835223], dtype=float32), array([0.90625], dtype=float32)]
Step 263, Epoch 19 Metrics [array([0.07475255], dtype=float32), array([0.96875], dtype=float32)]
Step 264, Epoch 19 Metrics [array([0.08070116], dtype=float32), array([0.953125], dtype=float32)]
Step 265, Epoch 19 Metrics [array([0.16567203], dtype=float32), array([0.953125], dtype=float32)]
Step 266, Epoch 19 Metrics [array([0.15193683], dtype=float32), array([0.9375], dtype=float32)]
Step 267, Epoch 19 Metrics [array([0.13525967], dtype=float32), array([0.953125], dtype=float32)]
Step 268, Epoch 19 Metrics [arr

Step 343, Epoch 19 Metrics [array([0.15914656], dtype=float32), array([0.953125], dtype=float32)]
Step 344, Epoch 19 Metrics [array([0.26200286], dtype=float32), array([0.90625], dtype=float32)]
Step 345, Epoch 19 Metrics [array([0.1960155], dtype=float32), array([0.890625], dtype=float32)]
Step 346, Epoch 19 Metrics [array([0.21581578], dtype=float32), array([0.921875], dtype=float32)]
Step 347, Epoch 19 Metrics [array([0.2396493], dtype=float32), array([0.9375], dtype=float32)]
Step 348, Epoch 19 Metrics [array([0.15813258], dtype=float32), array([0.90625], dtype=float32)]
Step 349, Epoch 19 Metrics [array([0.11244652], dtype=float32), array([0.984375], dtype=float32)]
Step 350, Epoch 19 Metrics [array([0.08506817], dtype=float32), array([0.953125], dtype=float32)]
Step 351, Epoch 19 Metrics [array([0.1065283], dtype=float32), array([0.96875], dtype=float32)]
Step 352, Epoch 19 Metrics [array([0.06202701], dtype=float32), array([0.984375], dtype=float32)]
Step 353, Epoch 19 Metrics [

In [13]:
train_loop(fluid.default_main_program())

Step 0, Epoch 0 Metrics [array([0.6787807], dtype=float32), array([0.625], dtype=float32)]
Step 1, Epoch 0 Metrics [array([0.6843128], dtype=float32), array([0.578125], dtype=float32)]
Step 2, Epoch 0 Metrics [array([0.69586754], dtype=float32), array([0.5], dtype=float32)]
Step 3, Epoch 0 Metrics [array([0.70143324], dtype=float32), array([0.46875], dtype=float32)]
Step 4, Epoch 0 Metrics [array([0.70037454], dtype=float32), array([0.46875], dtype=float32)]
Step 5, Epoch 0 Metrics [array([0.6714214], dtype=float32), array([0.640625], dtype=float32)]
Step 6, Epoch 0 Metrics [array([0.6975595], dtype=float32), array([0.5], dtype=float32)]
Step 7, Epoch 0 Metrics [array([0.6954325], dtype=float32), array([0.5], dtype=float32)]
Step 8, Epoch 0 Metrics [array([0.67823243], dtype=float32), array([0.609375], dtype=float32)]
Step 9, Epoch 0 Metrics [array([0.7013754], dtype=float32), array([0.46875], dtype=float32)]
Step 10, Epoch 0 Metrics [array([0.69374496], dtype=float32), array([0.5], dt

Step 88, Epoch 0 Metrics [array([0.68484527], dtype=float32), array([0.625], dtype=float32)]
Step 89, Epoch 0 Metrics [array([0.6863079], dtype=float32), array([0.625], dtype=float32)]
Step 90, Epoch 0 Metrics [array([0.68381554], dtype=float32), array([0.71875], dtype=float32)]
Step 91, Epoch 0 Metrics [array([0.6843819], dtype=float32), array([0.640625], dtype=float32)]
Step 92, Epoch 0 Metrics [array([0.6870451], dtype=float32), array([0.578125], dtype=float32)]
Step 93, Epoch 0 Metrics [array([0.68726426], dtype=float32), array([0.609375], dtype=float32)]
Step 94, Epoch 0 Metrics [array([0.68639296], dtype=float32), array([0.65625], dtype=float32)]
Step 95, Epoch 0 Metrics [array([0.68733215], dtype=float32), array([0.5625], dtype=float32)]
Step 96, Epoch 0 Metrics [array([0.68651927], dtype=float32), array([0.65625], dtype=float32)]
Step 97, Epoch 0 Metrics [array([0.684385], dtype=float32), array([0.578125], dtype=float32)]
Step 98, Epoch 0 Metrics [array([0.68000674], dtype=floa

Step 177, Epoch 0 Metrics [array([0.6752735], dtype=float32), array([0.671875], dtype=float32)]
Step 178, Epoch 0 Metrics [array([0.6758324], dtype=float32), array([0.703125], dtype=float32)]
Step 179, Epoch 0 Metrics [array([0.662427], dtype=float32), array([0.796875], dtype=float32)]
Step 180, Epoch 0 Metrics [array([0.6666446], dtype=float32), array([0.6875], dtype=float32)]
Step 181, Epoch 0 Metrics [array([0.6677413], dtype=float32), array([0.796875], dtype=float32)]
Step 182, Epoch 0 Metrics [array([0.66923755], dtype=float32), array([0.734375], dtype=float32)]
Step 183, Epoch 0 Metrics [array([0.6629769], dtype=float32), array([0.78125], dtype=float32)]
Step 184, Epoch 0 Metrics [array([0.6558678], dtype=float32), array([0.890625], dtype=float32)]
Step 185, Epoch 0 Metrics [array([0.6576607], dtype=float32), array([0.828125], dtype=float32)]
Step 186, Epoch 0 Metrics [array([0.6661715], dtype=float32), array([0.71875], dtype=float32)]
Step 187, Epoch 0 Metrics [array([0.6692277]

Step 265, Epoch 0 Metrics [array([0.6313077], dtype=float32), array([0.890625], dtype=float32)]
Step 266, Epoch 0 Metrics [array([0.6350969], dtype=float32), array([0.828125], dtype=float32)]
Step 267, Epoch 0 Metrics [array([0.6517713], dtype=float32), array([0.71875], dtype=float32)]
Step 268, Epoch 0 Metrics [array([0.6433159], dtype=float32), array([0.75], dtype=float32)]
Step 269, Epoch 0 Metrics [array([0.661911], dtype=float32), array([0.6875], dtype=float32)]
Step 270, Epoch 0 Metrics [array([0.6474018], dtype=float32), array([0.75], dtype=float32)]
Step 271, Epoch 0 Metrics [array([0.64038944], dtype=float32), array([0.78125], dtype=float32)]
Step 272, Epoch 0 Metrics [array([0.6378921], dtype=float32), array([0.75], dtype=float32)]
Step 273, Epoch 0 Metrics [array([0.63341117], dtype=float32), array([0.828125], dtype=float32)]
Step 274, Epoch 0 Metrics [array([0.65616405], dtype=float32), array([0.640625], dtype=float32)]
Step 275, Epoch 0 Metrics [array([0.63505864], dtype=f

Step 352, Epoch 0 Metrics [array([0.6304631], dtype=float32), array([0.734375], dtype=float32)]
Step 353, Epoch 0 Metrics [array([0.62318486], dtype=float32), array([0.71875], dtype=float32)]
Step 354, Epoch 0 Metrics [array([0.5804084], dtype=float32), array([0.8125], dtype=float32)]
Step 355, Epoch 0 Metrics [array([0.58274305], dtype=float32), array([0.828125], dtype=float32)]
Step 356, Epoch 0 Metrics [array([0.60994625], dtype=float32), array([0.75], dtype=float32)]
Step 357, Epoch 0 Metrics [array([0.60860825], dtype=float32), array([0.71875], dtype=float32)]
Step 358, Epoch 0 Metrics [array([0.6126785], dtype=float32), array([0.75], dtype=float32)]
Step 359, Epoch 0 Metrics [array([0.63825154], dtype=float32), array([0.671875], dtype=float32)]
Step 360, Epoch 0 Metrics [array([0.6192379], dtype=float32), array([0.671875], dtype=float32)]
Step 361, Epoch 0 Metrics [array([0.58906126], dtype=float32), array([0.765625], dtype=float32)]
Step 362, Epoch 0 Metrics [array([0.6048175], 

...

Step 376, Epoch 18 Metrics [array([0.1255306], dtype=float32), array([0.9375], dtype=float32)]
Step 377, Epoch 18 Metrics [array([0.06583016], dtype=float32), array([0.984375], dtype=float32)]
Step 378, Epoch 18 Metrics [array([0.12171265], dtype=float32), array([0.953125], dtype=float32)]
Step 379, Epoch 18 Metrics [array([0.10875688], dtype=float32), array([0.984375], dtype=float32)]
Step 380, Epoch 18 Metrics [array([0.06036053], dtype=float32), array([1.], dtype=float32)]
Step 381, Epoch 18 Metrics [array([0.11275941], dtype=float32), array([0.984375], dtype=float32)]
Step 382, Epoch 18 Metrics [array([0.09350418], dtype=float32), array([0.96875], dtype=float32)]
Step 383, Epoch 18 Metrics [array([0.03089913], dtype=float32), array([1.], dtype=float32)]
Step 384, Epoch 18 Metrics [array([0.11911631], dtype=float32), array([0.953125], dtype=float32)]
Step 385, Epoch 18 Metrics [array([0.11176771], dtype=float32), array([0.9375], dtype=float32)]
Step 386, Epoch 18 Metrics [array([0.0

Step 73, Epoch 19 Metrics [array([0.12570809], dtype=float32), array([0.984375], dtype=float32)]
Step 74, Epoch 19 Metrics [array([0.0589737], dtype=float32), array([1.], dtype=float32)]
Step 75, Epoch 19 Metrics [array([0.13350725], dtype=float32), array([0.96875], dtype=float32)]
Step 76, Epoch 19 Metrics [array([0.07770877], dtype=float32), array([1.], dtype=float32)]
Step 77, Epoch 19 Metrics [array([0.06617152], dtype=float32), array([0.984375], dtype=float32)]
Step 78, Epoch 19 Metrics [array([0.1468396], dtype=float32), array([0.9375], dtype=float32)]
Step 79, Epoch 19 Metrics [array([0.07117233], dtype=float32), array([0.984375], dtype=float32)]
Step 80, Epoch 19 Metrics [array([0.18049045], dtype=float32), array([0.9375], dtype=float32)]
Step 81, Epoch 19 Metrics [array([0.09994188], dtype=float32), array([0.953125], dtype=float32)]
Step 82, Epoch 19 Metrics [array([0.14821425], dtype=float32), array([0.96875], dtype=float32)]
Step 83, Epoch 19 Metrics [array([0.08498499], dty

Step 160, Epoch 19 Metrics [array([0.21368252], dtype=float32), array([0.9375], dtype=float32)]
Step 161, Epoch 19 Metrics [array([0.05907945], dtype=float32), array([0.984375], dtype=float32)]
Step 162, Epoch 19 Metrics [array([0.06602105], dtype=float32), array([0.984375], dtype=float32)]
Step 163, Epoch 19 Metrics [array([0.09699109], dtype=float32), array([0.953125], dtype=float32)]
Step 164, Epoch 19 Metrics [array([0.17010994], dtype=float32), array([0.921875], dtype=float32)]
Step 165, Epoch 19 Metrics [array([0.09022934], dtype=float32), array([0.953125], dtype=float32)]
Step 166, Epoch 19 Metrics [array([0.14939731], dtype=float32), array([0.953125], dtype=float32)]
Step 167, Epoch 19 Metrics [array([0.07466845], dtype=float32), array([1.], dtype=float32)]
Step 168, Epoch 19 Metrics [array([0.13349943], dtype=float32), array([0.953125], dtype=float32)]
Step 169, Epoch 19 Metrics [array([0.14932588], dtype=float32), array([0.9375], dtype=float32)]
Step 170, Epoch 19 Metrics [ar

Step 246, Epoch 19 Metrics [array([0.05949449], dtype=float32), array([0.984375], dtype=float32)]
Step 247, Epoch 19 Metrics [array([0.07666157], dtype=float32), array([0.984375], dtype=float32)]
Step 248, Epoch 19 Metrics [array([0.11909713], dtype=float32), array([0.96875], dtype=float32)]
Step 249, Epoch 19 Metrics [array([0.09799183], dtype=float32), array([0.984375], dtype=float32)]
Step 250, Epoch 19 Metrics [array([0.12093493], dtype=float32), array([0.984375], dtype=float32)]
Step 251, Epoch 19 Metrics [array([0.0624404], dtype=float32), array([1.], dtype=float32)]
Step 252, Epoch 19 Metrics [array([0.13795061], dtype=float32), array([0.9375], dtype=float32)]
Step 253, Epoch 19 Metrics [array([0.15248607], dtype=float32), array([0.9375], dtype=float32)]
Step 254, Epoch 19 Metrics [array([0.10196312], dtype=float32), array([0.953125], dtype=float32)]
Step 255, Epoch 19 Metrics [array([0.14289585], dtype=float32), array([0.9375], dtype=float32)]
Step 256, Epoch 19 Metrics [array(

Step 333, Epoch 19 Metrics [array([0.14166136], dtype=float32), array([0.953125], dtype=float32)]
Step 334, Epoch 19 Metrics [array([0.05218843], dtype=float32), array([0.984375], dtype=float32)]
Step 335, Epoch 19 Metrics [array([0.08720121], dtype=float32), array([0.96875], dtype=float32)]
Step 336, Epoch 19 Metrics [array([0.13134322], dtype=float32), array([0.96875], dtype=float32)]
Step 337, Epoch 19 Metrics [array([0.03488965], dtype=float32), array([1.], dtype=float32)]
Step 338, Epoch 19 Metrics [array([0.22273397], dtype=float32), array([0.953125], dtype=float32)]
Step 339, Epoch 19 Metrics [array([0.12248366], dtype=float32), array([0.96875], dtype=float32)]
Step 340, Epoch 19 Metrics [array([0.11355143], dtype=float32), array([0.953125], dtype=float32)]
Step 341, Epoch 19 Metrics [array([0.16182396], dtype=float32), array([0.96875], dtype=float32)]
Step 342, Epoch 19 Metrics [array([0.19539094], dtype=float32), array([0.9375], dtype=float32)]
Step 343, Epoch 19 Metrics [arra

In [14]:
train_loop(fluid.default_main_program())

Step 0, Epoch 0 Metrics [array([0.70252347], dtype=float32), array([0.46875], dtype=float32)]
Step 1, Epoch 0 Metrics [array([0.6956351], dtype=float32), array([0.515625], dtype=float32)]
Step 2, Epoch 0 Metrics [array([0.7047142], dtype=float32), array([0.46875], dtype=float32)]
Step 3, Epoch 0 Metrics [array([0.6896099], dtype=float32), array([0.546875], dtype=float32)]
Step 4, Epoch 0 Metrics [array([0.7091137], dtype=float32), array([0.4375], dtype=float32)]
Step 5, Epoch 0 Metrics [array([0.70426506], dtype=float32), array([0.46875], dtype=float32)]
Step 6, Epoch 0 Metrics [array([0.68375313], dtype=float32), array([0.578125], dtype=float32)]
Step 7, Epoch 0 Metrics [array([0.717237], dtype=float32), array([0.375], dtype=float32)]
Step 8, Epoch 0 Metrics [array([0.69430995], dtype=float32), array([0.5], dtype=float32)]
Step 9, Epoch 0 Metrics [array([0.66746336], dtype=float32), array([0.671875], dtype=float32)]
Step 10, Epoch 0 Metrics [array([0.6964823], dtype=float32), array([0

Step 91, Epoch 0 Metrics [array([0.68406177], dtype=float32), array([0.65625], dtype=float32)]
Step 92, Epoch 0 Metrics [array([0.68118155], dtype=float32), array([0.765625], dtype=float32)]
Step 93, Epoch 0 Metrics [array([0.68273205], dtype=float32), array([0.71875], dtype=float32)]
Step 94, Epoch 0 Metrics [array([0.68072695], dtype=float32), array([0.734375], dtype=float32)]
Step 95, Epoch 0 Metrics [array([0.6824742], dtype=float32), array([0.734375], dtype=float32)]
Step 96, Epoch 0 Metrics [array([0.68268067], dtype=float32), array([0.6875], dtype=float32)]
Step 97, Epoch 0 Metrics [array([0.6822205], dtype=float32), array([0.71875], dtype=float32)]
Step 98, Epoch 0 Metrics [array([0.6804697], dtype=float32), array([0.828125], dtype=float32)]
Step 99, Epoch 0 Metrics [array([0.6791103], dtype=float32), array([0.796875], dtype=float32)]
Step 100, Epoch 0 Metrics [array([0.68221414], dtype=float32), array([0.6875], dtype=float32)]
Step 101, Epoch 0 Metrics [array([0.6841019], dtyp

Step 178, Epoch 0 Metrics [array([0.6639067], dtype=float32), array([0.78125], dtype=float32)]
Step 179, Epoch 0 Metrics [array([0.66108847], dtype=float32), array([0.78125], dtype=float32)]
Step 180, Epoch 0 Metrics [array([0.6676589], dtype=float32), array([0.71875], dtype=float32)]
Step 181, Epoch 0 Metrics [array([0.66395926], dtype=float32), array([0.8125], dtype=float32)]
Step 182, Epoch 0 Metrics [array([0.6671903], dtype=float32), array([0.75], dtype=float32)]
Step 183, Epoch 0 Metrics [array([0.6559083], dtype=float32), array([0.859375], dtype=float32)]
Step 184, Epoch 0 Metrics [array([0.658863], dtype=float32), array([0.765625], dtype=float32)]
Step 185, Epoch 0 Metrics [array([0.65932304], dtype=float32), array([0.796875], dtype=float32)]
Step 186, Epoch 0 Metrics [array([0.6625025], dtype=float32), array([0.78125], dtype=float32)]
Step 187, Epoch 0 Metrics [array([0.662993], dtype=float32), array([0.6875], dtype=float32)]
Step 188, Epoch 0 Metrics [array([0.6569989], dtype

Step 266, Epoch 0 Metrics [array([0.6369587], dtype=float32), array([0.765625], dtype=float32)]
Step 267, Epoch 0 Metrics [array([0.6060856], dtype=float32), array([0.890625], dtype=float32)]
Step 268, Epoch 0 Metrics [array([0.6290164], dtype=float32), array([0.734375], dtype=float32)]
Step 269, Epoch 0 Metrics [array([0.6415294], dtype=float32), array([0.796875], dtype=float32)]
Step 270, Epoch 0 Metrics [array([0.6342727], dtype=float32), array([0.765625], dtype=float32)]
Step 271, Epoch 0 Metrics [array([0.6307242], dtype=float32), array([0.796875], dtype=float32)]
Step 272, Epoch 0 Metrics [array([0.63793594], dtype=float32), array([0.78125], dtype=float32)]
Step 273, Epoch 0 Metrics [array([0.62669605], dtype=float32), array([0.765625], dtype=float32)]
Step 274, Epoch 0 Metrics [array([0.6296849], dtype=float32), array([0.75], dtype=float32)]
Step 275, Epoch 0 Metrics [array([0.63305664], dtype=float32), array([0.75], dtype=float32)]
Step 276, Epoch 0 Metrics [array([0.61592317],

Step 353, Epoch 0 Metrics [array([0.5827891], dtype=float32), array([0.859375], dtype=float32)]
Step 354, Epoch 0 Metrics [array([0.5863031], dtype=float32), array([0.796875], dtype=float32)]
Step 355, Epoch 0 Metrics [array([0.59967357], dtype=float32), array([0.75], dtype=float32)]
Step 356, Epoch 0 Metrics [array([0.5853249], dtype=float32), array([0.84375], dtype=float32)]
Step 357, Epoch 0 Metrics [array([0.602514], dtype=float32), array([0.75], dtype=float32)]
Step 358, Epoch 0 Metrics [array([0.5657526], dtype=float32), array([0.859375], dtype=float32)]
Step 359, Epoch 0 Metrics [array([0.5890271], dtype=float32), array([0.734375], dtype=float32)]
Step 360, Epoch 0 Metrics [array([0.5822351], dtype=float32), array([0.859375], dtype=float32)]
Step 361, Epoch 0 Metrics [array([0.5898129], dtype=float32), array([0.765625], dtype=float32)]
Step 362, Epoch 0 Metrics [array([0.5839188], dtype=float32), array([0.75], dtype=float32)]
Step 363, Epoch 0 Metrics [array([0.58519125], dtype=

...

Step 5, Epoch 19 Metrics [array([0.13540864], dtype=float32), array([0.953125], dtype=float32)]
Step 6, Epoch 19 Metrics [array([0.1794668], dtype=float32), array([0.921875], dtype=float32)]
Step 7, Epoch 19 Metrics [array([0.1443333], dtype=float32), array([0.953125], dtype=float32)]
Step 8, Epoch 19 Metrics [array([0.05124136], dtype=float32), array([1.], dtype=float32)]
Step 9, Epoch 19 Metrics [array([0.13224807], dtype=float32), array([0.984375], dtype=float32)]
Step 10, Epoch 19 Metrics [array([0.07938667], dtype=float32), array([0.96875], dtype=float32)]
Step 11, Epoch 19 Metrics [array([0.16519764], dtype=float32), array([0.9375], dtype=float32)]
Step 12, Epoch 19 Metrics [array([0.11974193], dtype=float32), array([0.953125], dtype=float32)]
Step 13, Epoch 19 Metrics [array([0.1332238], dtype=float32), array([0.953125], dtype=float32)]
Step 14, Epoch 19 Metrics [array([0.17977877], dtype=float32), array([0.921875], dtype=float32)]
Step 15, Epoch 19 Metrics [array([0.05114952], 

Step 91, Epoch 19 Metrics [array([0.12669286], dtype=float32), array([0.953125], dtype=float32)]
Step 92, Epoch 19 Metrics [array([0.08356923], dtype=float32), array([0.96875], dtype=float32)]
Step 93, Epoch 19 Metrics [array([0.13161105], dtype=float32), array([0.984375], dtype=float32)]
Step 94, Epoch 19 Metrics [array([0.04198408], dtype=float32), array([1.], dtype=float32)]
Step 95, Epoch 19 Metrics [array([0.09690404], dtype=float32), array([0.953125], dtype=float32)]
Step 96, Epoch 19 Metrics [array([0.12873562], dtype=float32), array([0.953125], dtype=float32)]
Step 97, Epoch 19 Metrics [array([0.08772226], dtype=float32), array([0.953125], dtype=float32)]
Step 98, Epoch 19 Metrics [array([0.08252104], dtype=float32), array([0.984375], dtype=float32)]
Step 99, Epoch 19 Metrics [array([0.11681774], dtype=float32), array([0.96875], dtype=float32)]
Step 100, Epoch 19 Metrics [array([0.20624335], dtype=float32), array([0.921875], dtype=float32)]
Step 101, Epoch 19 Metrics [array([0.

Step 176, Epoch 19 Metrics [array([0.2721797], dtype=float32), array([0.921875], dtype=float32)]
Step 177, Epoch 19 Metrics [array([0.12434585], dtype=float32), array([0.96875], dtype=float32)]
Step 178, Epoch 19 Metrics [array([0.11615912], dtype=float32), array([0.9375], dtype=float32)]
Step 179, Epoch 19 Metrics [array([0.06806532], dtype=float32), array([1.], dtype=float32)]
Step 180, Epoch 19 Metrics [array([0.09279953], dtype=float32), array([0.96875], dtype=float32)]
Step 181, Epoch 19 Metrics [array([0.12008633], dtype=float32), array([0.953125], dtype=float32)]
Step 182, Epoch 19 Metrics [array([0.16488656], dtype=float32), array([0.9375], dtype=float32)]
Step 183, Epoch 19 Metrics [array([0.27196822], dtype=float32), array([0.9375], dtype=float32)]
Step 184, Epoch 19 Metrics [array([0.11003648], dtype=float32), array([0.953125], dtype=float32)]
Step 185, Epoch 19 Metrics [array([0.09895019], dtype=float32), array([0.9375], dtype=float32)]
Step 186, Epoch 19 Metrics [array([0.

Step 262, Epoch 19 Metrics [array([0.14838609], dtype=float32), array([0.953125], dtype=float32)]
Step 263, Epoch 19 Metrics [array([0.13516298], dtype=float32), array([0.9375], dtype=float32)]
Step 264, Epoch 19 Metrics [array([0.09635004], dtype=float32), array([0.984375], dtype=float32)]
Step 265, Epoch 19 Metrics [array([0.14325646], dtype=float32), array([0.9375], dtype=float32)]
Step 266, Epoch 19 Metrics [array([0.07089058], dtype=float32), array([0.984375], dtype=float32)]
Step 267, Epoch 19 Metrics [array([0.14017135], dtype=float32), array([0.96875], dtype=float32)]
Step 268, Epoch 19 Metrics [array([0.18125859], dtype=float32), array([0.953125], dtype=float32)]
Step 269, Epoch 19 Metrics [array([0.12378213], dtype=float32), array([0.984375], dtype=float32)]
Step 270, Epoch 19 Metrics [array([0.21214995], dtype=float32), array([0.921875], dtype=float32)]
Step 271, Epoch 19 Metrics [array([0.15160881], dtype=float32), array([0.96875], dtype=float32)]
Step 272, Epoch 19 Metrics

Step 347, Epoch 19 Metrics [array([0.12029868], dtype=float32), array([0.9375], dtype=float32)]
Step 348, Epoch 19 Metrics [array([0.17632452], dtype=float32), array([0.9375], dtype=float32)]
Step 349, Epoch 19 Metrics [array([0.17632794], dtype=float32), array([0.9375], dtype=float32)]
Step 350, Epoch 19 Metrics [array([0.1645785], dtype=float32), array([0.953125], dtype=float32)]
Step 351, Epoch 19 Metrics [array([0.1008991], dtype=float32), array([0.984375], dtype=float32)]
Step 352, Epoch 19 Metrics [array([0.11119559], dtype=float32), array([0.953125], dtype=float32)]
Step 353, Epoch 19 Metrics [array([0.10638051], dtype=float32), array([0.984375], dtype=float32)]
Step 354, Epoch 19 Metrics [array([0.1282736], dtype=float32), array([0.953125], dtype=float32)]
Step 355, Epoch 19 Metrics [array([0.13539842], dtype=float32), array([0.953125], dtype=float32)]
Step 356, Epoch 19 Metrics [array([0.08175893], dtype=float32), array([0.984375], dtype=float32)]
Step 357, Epoch 19 Metrics [a